In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Simulation parameters
N = 100         # number of particles
L = 10.0             # box size (0..L in both x and y)
dt = 0.05        # time step


# Initial state
pos=np.random.rand(N,3)*L
vel=np.random.uniform(-1,1,size=(N,3))*2
plt.rcParams['animation.embed_limit'] = 500

mass=1
radius  = 0.1

k = 1000
r0 = 1.0
rc = 2.0


In [ ]:
def pair_forces(pos):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    mask = (r < rc) & (r > 1e-12)
    u = d / (r[:, :, None] + 1e-12)
    fmag = -1*k * (r - r0) * mask
    F = np.sum(fmag[:, :, None] * u, axis=1)
    return F

In [ ]:
def walls_reflect_other(pos, vel, L, x, y,z):


    vel[x, 0] *= -1
    pos[x, 0] = np.where(pos[x, 0] < 0, 0.0, np.where(pos[x, 0] > L, L, pos[x, 0]))


    vel[y, 1] *= -1
    pos[y, 1] = np.where(pos[y, 1] < 0, 0.0, np.where(pos[y, 1] > L, L, pos[y, 1]))

    vel[z, 2] *= -1
    pos[z, 2] = np.where(pos[z, 2] < 0, 0.0, np.where(pos[z, 2] > L, L, pos[z, 2]))


In [ ]:
def step_smooth(pos,vel,mass,dt,L,F_prev):

    if F_prev is None:
        F_prev = pair_forces(pos)
    a_prev = F_prev / mass

    vel = vel + 0.5 * a_prev * dt

    pos = pos + vel * dt
    x = (pos[:, 0] < 0) | (pos[:, 0] > L)
    y = (pos[:, 1] < 0) | (pos[:, 1] > L)
    z = (pos[:, 2] < 0) | (pos[:, 2] > L)
    walls_reflect_other(pos, vel, L, x, y,z)

    # new forces
    F_new = pair_forces(pos)
    a_new = F_new / mass

    vel = vel + 0.5 * a_new * dt


    return pos, vel, F_new


In [ ]:
def kinetic_energy(vel, mass):
    return 0.5 * mass * (np.linalg.norm(vel) ** 2)

def potential_energy(pos, k=1.0, r0=0.8, rc=2.0):
    d = pos[None, :, :] - pos[:, None, :]       # pairwise displacement
    r = np.linalg.norm(d, axis=2)               # pairwise distance
    mask_valid = (r > 1e-12)                    # avoid self-interactions
    mask_inside = (r < rc) & mask_valid         # within cutoff
    U = -0.5 * k * (r - r0)**2 * mask_inside
    U += (-0.5 * k * (rc - r0)**2) * ((r >= rc) & mask_valid)
    # Divide by 2 to avoid double-counting pairs
    PE = np.sum(U) / 2.0
    return PE

In [ ]:
def detect_collisions(pos, radius):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    colliding_pairs = np.array(np.where((r < 2 * radius) & (r > 1e-12))).T
    colliding_pairs = colliding_pairs[colliding_pairs[:,0] < colliding_pairs[:,1]]
    return colliding_pairs

def resolve_collision(pos, vel, i, j):
    r_rel = pos[i] - pos[j]
    dist = np.linalg.norm(r_rel)
    if dist == 0:
        return

    n = r_rel / dist

    v_i_n = np.dot(vel[i], n)
    v_j_n = np.dot(vel[j], n)

    vel[i] += (v_j_n - v_i_n) * n
    vel[j] += (v_i_n - v_j_n) * n

def step_with_collisions(pos, vel, mass, dt, L, F_prev, radius):
    pos, vel, F_new = step_smooth(pos, vel, mass, dt, L, F_prev)
    collisions = detect_collisions(pos, radius)
    for i, j in collisions:
            resolve_collision(pos, vel, i, j)
    return pos, vel, F_new

In [ ]:
plt.close('all')
fig = plt.figure(figsize = (8,4) , dpi = 200)
ax  = fig.add_subplot(121,projection='3d')
ax_energy =  fig.add_subplot(122)
plt.subplots_adjust(wspace=0.6)
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_zlim(0, L)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Particles (step_smooth)')

ax_energy.set_xlabel("Time(Seconds)")
ax_energy.set_ylabel("Energy(Joules)")
ax_energy.set_title("Energy vs Time")

ke_line, = ax_energy.plot([], [], color='tab:red', label="KE")
pe_line, = ax_energy.plot([], [], color='tab:blue', label="PE")
te_line, = ax_energy.plot([], [], color='tab:green', label="TE")
ax_energy.legend(loc="upper left", bbox_to_anchor=(1.02, 1))
ke_times, ke_values, pe_values, te_values = [], [], [], []
delta_ke = ax_energy.text(0.02, 0.95, '', transform=ax_energy.transAxes, fontsize=12, color='blue')
ke_max_ss = 0
ke_count_ss = 0
ke_sum_ss = 0


scatter = ax.scatter([], [], [], s=60, c='tab:blue', alpha=0.8)
F_prev = None
def init():
    scatter._offsets3d=(pos[:,0],pos[:,1],pos[:,2])
    ke_line.set_data([], [])
    pe_line.set_data([], [])
    te_line.set_data([], [])
    ke_times.clear()
    ke_values.clear()
    pe_values.clear()
    te_values.clear()
    return scatter, ke_line, pe_line, te_line

delta_ke = ax_energy.text(0.02, 0.95, '', transform=ax_energy.transAxes, fontsize=6, color='blue')

def animate(frame):
    global pos, vel, ke_max_ss, ke_sum_ss, ke_count_ss, F_prev
    pos, vel, F_prev = step_smooth(pos, vel, mass, dt, L, F_prev,radius)
    scatter._offsets3d=(pos[:,0],pos[:,1],pos[:,2])
    t = (frame+1)*dt
    KE = kinetic_energy(vel, mass)
    PE = potential_energy(pos, k, r0, rc)
    TE = KE + PE

    ke_times.append(t)
    ke_values.append(KE)
    pe_values.append(PE)
    te_values.append(TE)

    ke_line.set_data(ke_times, ke_values)
    pe_line.set_data(ke_times, pe_values)
    te_line.set_data(ke_times, te_values)

    ax_energy.set_xlim(0, max(dt, t))
    ax_energy.set_xlim(0, max(dt, t))
    ax_energy.set_ylim(min(pe_values)*2, max(ke_values)*2)


    if len(te_values) > 1 and te_values[0] != 0:
        delta_ke.set_text(f"STD: TE {np.std(te_values):.1f}  STD: KE {np.std(ke_values):.1f}  STD: PE {np.std(pe_values):.1f}")
    else:
        delta_ke.set_text("ΔTE%: 0.00%")

    return scatter, ke_line, pe_line, te_line, delta_ke

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=10,blit = True)

with open("3D_collisions.html", "w") as f:
    f.write(anim.to_jshtml())